In [1]:
# https://github.com/karthikvijayakumar/Kaggle/blob/master/AirBnB_Seattle/Seattle_AirBnB_exploration.ipynb
# https://www.citypopulation.de/php/ireland-dublin.php
# https://data.gov.ie/dataset/dublin-city-roads-and-streets
# https://www.geopostcodes.com/Dublin_10_Dublin

In [2]:
# !pip install sklearn
# !pip install fuzzywuzzy

In [3]:
import pandas as pd
import numpy as np
import csv
import re
# from fuzzywuzzy import fuzz


pd.set_option('display.max_colwidth', -1)

In [4]:
%cd ../../data/airbnbdata
# %cd ../
!ls

/usr/local/bin/notebooks/data/airbnbdata
201702_calendar.csv	      201702_reviews.csv	 201811_reviews.csv
201702_calendar.csv.gz	      201702_reviews.csv.gz	 201811_reviews.csv.gz
201702_listings.csv	      201811_calendar.csv	 column_names.txt
201702_listings.csv.gz	      201811_calendar.csv.gz	 core
201702_listings_geocoded.csv  201811_listings.csv	 geo
201702_listings_preproc.csv   201811_listings.csv.gz	 housing
201702_neighbourhoods.csv     201811_neighbourhoods.csv


In [5]:
# filename = '201702_listings.csv.gz' # 94 cols
filename = '201702_listings_preproc.csv' # 15 cols

df_read = pd.read_csv(filename,
                       encoding='utf-8',
#                        compression='gzip',
                       sep=',',
                       index_col = 'host_id',
                       header=0
                       )

print(df_read.shape)
df_read.head(1)
# df_read.columns

(5376, 93)


,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
host_id,,,,,,,,,,,,,,,,,,,,,
51820708,https://www.airbnb.com/rooms/10778114,20170218121908,2017-02-18,1 Bedroom Apartment For Short Term,Contact for more info,- Kitchen - Wifi - PS4,"Contact for more info - Kitchen - Wifi - PS4 1 minute from DART, dublin bikes, and bus stop",none,NaN,NaN,...,NaN,f,NaN,NaN,f,flexible,f,f,1,NaN


In [6]:
filename = '201702_reviews.csv.gz' # 15 cols
reviews = pd.read_csv(filename,
                       encoding='latin-1',
                       compression='gzip',
                       sep=',',
#                        index_col = 'host_id',
                       header=0
                       )

print(reviews.shape)
# streets = streets[['street_name', 'new_area_code']]
# streets['street_name'] = streets['street_name'].str.lower()
reviews.head(1)

(141152, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2067000,22332759,2014-11-03,21552104,Valentina,"We felt like home. John is a pefect host, we got lost and he came to show us the right way to reach his house. The room was big and clean and the surrounding area safe. The house is just 30 mins walking from the city center."


#### Clean Street Adress data

In [7]:
# d1|d2|d3|d4|d5|d6|d6W|d7|d8|d9|d10|
# d11|d12|d13|d14|d15|d16|d17|d18|d19|d20|
# d21|d22|d23|d24|
# Dublin 1|Dublin 2|Dublin 3|Dublin 4|Dublin 5|Dublin 6|Dublin 6W|Dublin 7|Dublin 8|Dublin 9|Dublin 10|
# Dublin 11|Dublin 12|Dublin 13|Dublin 14|Dublin 15|Dublin 16|Dublin 17|Dublin 18|Dublin 19|Dublin 20|
# Dublin 21|Dublin 22|Dublin 23|Dublin 24|


In [8]:
pattern = '''
Arran Quay|Ashtown|Ayrfield|
Balbriggan|Baldoyle|Balgriffin|Ballinascorney|Ballinteer|Ballyboden|Ballyboghil|Ballybough|Ballybrack|Ballygall|Ballymun|Balscadden|Beaumont|Blackrock|Blanchardstown|Bohernabreena|Botanic|
Cabinteely|Cabra|Carna|Castleknock|Chapelizod|Cherrywood|Churchtown|Clondalkin|Clonmethan|Clonskeagh|Clontarf|Crumlin|
Dalkey|Decies|Donabate|Drumcondra|Drumfinn|Dubber|Dun Laoghaire|Dundrum|
Edenmore|Edmondstown|
Finglas|Firhouse|Foxrock|
Garristown|Glencullen|Grace|Grange|
Harmonstown|Hollywood|Holmpatrick|Howth|
Inchicore|Inns Quay|
Killiney|Kilmainham|Kilmore|Kilsallaghan|Kimmage|Kinsaley|Kylemore|
Lucan|Lusk|
Malahide|Mansion|Merchants Quay|Mountjoy|
Newcastle|North|
Palmerston|Pembroke|Phoenix|Portmarnock|Priorswood|
Raheny|Rathcoole|Rathfarnham|Rathmines|Rotunda|Royal Exchange|Rush|
Saggart|Shankill|Skerries|South Dock|St. Kevin's|Stillorgan|Sutton|Swords|
Tallaght|Templeogue|Terenure|The Ward|Tibradden|Turnapin|
Ushers|
Walkinstown|Whitehall|Wood Quay|
Abbey Street|Amiens Street| 
Arbour Hill|
Ashtown|
Ayrfield|
Baldoyle|
Balgriffin|
Ballinteer|
Ballsbridge|
Ballyboden|
Ballybough|
Ballyedmonduff|
Ballyfermot|
Ballymun|
Bayside|
Beaumont|
Belcamp|
Belfield|
Blanchardstown|
Bluebell|
Botanic Gardens|
Broadstone|
Cabinteely|
Cabra|
Capel Street|
Carrickmines|
Castleknock|
Chapelizod|
Cherry Orchard|
Churchtown|
Clondalkin|
Clonee|
Clonliffe|
Clonsilla|
Clonskeagh|
Clontarf|
Coolmine|
Coolock|
Coolquay|
Corduff|
Coultry|
Crumlin|
Dame Street|
Darndale|
Dartry|
Dollymount|
Dolphin's Barn|
Donaghmede|
Donnybrook|
Donnycarney|
Dorset Street| 
Drimnagh|
Drumcondra|
Dubber Cross|
Artane|
Dundrum|
East Point|
East Wall|
Elm Mount|
Fairview|
Finglas|
Firhouse|
Foxrock|
Glasnevin|
Goatstown|
Grafton Street|
Grangegorman|
Greenhills|
Griffith Avenue|
Harmonstown|
Harold's Cross|
Henry Street|
Howth|
Inchicore|
Irishtown|
Islandbridge|
Jobstownand|
Kilbarrackand|
Killester|
Kilmainham|
Kilmashogue|
Kilshane Cross|
Kilternan|
Kimmage|
Knocklyon|
Leeson Street|
Leopardstown|
Liberties|
Liffey Valley|
Marino|
Marlborough Street|
Mary Street|
Merchants Quay|
Merrion|
Merrion Square|
Met Eireann|
Milltown|
Mountjoy Square|
Mulhuddart|
Neilstown|
North Strand|
North Wall|
O'Connell Street|
Ongar|
Oxmantown|
Palmerstown|
Parnell Square|
Pembroke|
Perrystown|
Phibsboro|
Phoenix Park|
Portobello|
Priorswood|
Quarryvale|
Raheny|
Ranelagh|
Rathfarnham|
Rathgar|
Rathmines|
Ringsend|
Rockbrook|
Rowlagh|
Sandyford|
Sandymount|
Santry|
Shangan|
Shankill|
Smithfield|
South Circular Road|
St Mobhi|
Stepaside|
Stoneybatter|
Sutton|
Talbot Street|
Tallaght|
Temple Bar|
Templeogue|
Terenure|
The Ward|
Ticknock|
Tyrrelstown|
Walkinstown|
Wexford Street|
Whitehall|
Windy Arbour
'''.replace('\n','').lower()

In [9]:
pattern
type(pattern)

str

In [10]:
df = df_read.copy()
df = df[['street','zipcode', 'description', 'longitude', 'latitude']]

# text='South Lotts Road, Dublin, Dublin Dublin 4, Ireland'
# text = 'county dublin|d8|dublin 2'
# df['test'] = df['street'].str.extract('(Dublin [0-9])', expand=True)
# df['test'] = df['street'].str.extract('(Foxrock|County)', expand=True)

df['street'] = df['street'].str.lower()
df['zipcode'] = df['zipcode'].str.lower()
df['description'] = df['description'].str.lower()

df['street_cleansed'] = df['street'].str.extract(r'('+pattern+')', expand=True)
# df['street_cleansed'] = df['zipcode'].str.extract('('+pattern+')', expand=True)
df['street_cleansed_from_descr'] = df['description'].str.extract(r'('+pattern+')', expand=True)

# try using zipcode value lookup
mask = pd.isnull(df['street_cleansed'])
column_name = 'street_cleansed'
df['street_cleansed'] = np.where(mask, 
                                 df['zipcode'], 
                                 df['street_cleansed'])

# df.loc[[72253535]]
# df = df.loc[[27514157]]
# df
# print(len(df[~pd.isnull(df.street_cleansed_from_descr) & pd.isnull(df.street_cleansed)]))
# df[~pd.isnull(df.street_cleansed_from_descr) & pd.isnull(df.street_cleansed)].head()
# df.head()

In [11]:
# county dublin

print(len(df[df.street_cleansed=='county dublin']))
df[df.street_cleansed=='county dublin'].head()

0


,street,zipcode,description,longitude,latitude,street_cleansed,street_cleansed_from_descr
host_id,,,,,,,


#### mapping

In [12]:
# mapping
AreaDict  = {
'Abbey Street' : 'Dublin 1',
'Amiens Street' : 'Dublin 1',
'Capel Street' : 'Dublin 1',
'Dorset Street' : 'Dublin 1',
'Henry Street' : 'Dublin 1',
'Mary Street' : 'Dublin 1',
'Mountjoy Square' : 'Dublin 1',
'Marlborough Street' : 'Dublin 1',
'North Wall' : 'Dublin 1',
'O\'Connell Street' : 'Dublin 1',
'Parnell Square' : 'Dublin 1',
'Talbot Street' : 'Dublin 1',
'Merrion Square' : 'Dublin 2',
'Temple Bar' : 'Dublin 2',
'Grafton Street' : 'Dublin 2',
'Dame Street' : 'Dublin 2',
'Leeson Street' : 'Dublin 2',
'Wexford Street' : 'Dublin 2',
'Ballybough' : 'Dublin 3',
'North Strand' : 'Dublin 3',
'Clonliffe' : 'Dublin 3',
'Clontarf' : 'Dublin 3',
'Dollymount' : 'Dublin 3',
'East Wall' : 'Dublin 3',
'East Point' : 'Dublin 3',
'Fairview' : 'Dublin 3',
'Killester' : 'Dublin 3',
'Marino' : 'Dublin 3',
'Ballsbridge' : 'Dublin 4',
'Belfield' : 'Dublin 4',
'Donnybrook' : 'Dublin 4',
'Irishtown' : 'Dublin 4',
'Merrion' : 'Dublin 4',
'Pembroke' : 'Dublin 4',
'Ringsend' : 'Dublin 4',
'Sandymount' : 'Dublin 4',
'Artane' : 'Dublin 5',
'Coolock' : 'Dublin 5',
'Harmonstown' : 'Dublin 5',
'Kilbarrackand' : 'Dublin 5',
'Raheny' : 'Dublin 5',
'Milltown' : 'Dublin 6',
'Ranelagh' : 'Dublin 6',
'Rathmines' : 'Dublin 6',
'Dartry' : 'Dublin 6',
'Rathgar' : 'Dublin 6',
'Harold\'s Cross' : 'Dublin 6W',
'Templeogue' : 'Dublin 6W',
'Kimmage' : 'Dublin 6W',
'Terenure' : 'Dublin 6W',
'Arbour Hill' : 'Dublin 7',
'Ashtown' : 'Dublin 7',
'Broadstone' : 'Dublin 7',
'Cabra' : 'Dublin 7',
'Grangegorman' : 'Dublin 7',
'Oxmantown' : 'Dublin 7',
'Phibsboro' : 'Dublin 7',
'Smithfield' : 'Dublin 7',
'Stoneybatter' : 'Dublin 7',
'Dolphin\'s Barn' : 'Dublin 8',
'Inchicore' : 'Dublin 8',
'Islandbridge' : 'Dublin 8',
'Kilmainham' : 'Dublin 8',
'Merchants Quay' : 'Dublin 8',
'Portobello' : 'Dublin 8',
'South Circular Road' : 'Dublin 8',
'Phoenix Park' : 'Dublin 8',
'Liberties' : 'Dublin 8',
'Ballymun' : 'Dublin 9',
'Shangan' : 'Dublin 9',
'Coultry' : 'Dublin 9',
'Beaumont' : 'Dublin 9',
'Donnycarney' : 'Dublin 9',
'Drumcondra' : 'Dublin 9',
'Elm Mount' : 'Dublin 9',
'Griffith Avenue' : 'Dublin 9',
'Glasnevin' : 'Dublin 9',
'St Mobhi' : 'Dublin 9',
'Botanic Gardens' : 'Dublin 9',
'Met Eireann' : 'Dublin 9',
'Santry' : 'Dublin 9',
'Whitehall' : 'Dublin 9',
'Ballyfermot' : 'Dublin 10',
'Cherry Orchard' : 'Dublin 10',
'Ballymun' : 'Dublin 11',
'Dubber Cross' : 'Dublin 11',
'Finglas' : 'Dublin 11',
'Glasnevin' : 'Dublin 11',
'Kilshane Cross' : 'Dublin 11',
'The Ward' : 'Dublin 11', 
'Coolquay' : 'Dublin 11',
'Bluebell' : 'Dublin 12',
'Crumlin' : 'Dublin 12',
'Drimnagh' : 'Dublin 12',
'Greenhills' : 'Dublin 12',
'Perrystown' : 'Dublin 12',
'Terenure' : 'Dublin 12',
'Walkinstown' : 'Dublin 12',
'Baldoyle' : 'Dublin 13',
'Bayside' : 'Dublin 13',
'Donaghmede' : 'Dublin 13',
'Sutton' : 'Dublin 13',
'Howth' : 'Dublin 13',
'Ayrfield' : 'Dublin 13',
'Churchtown' : 'Dublin 14',
'Clonskeagh' : 'Dublin 14',
'Dundrum' : 'Dublin 14',
'Goatstown' : 'Dublin 14',
'Rathfarnham' : 'Dublin 14',
'Windy Arbour' : 'Dublin 14',
'Ashtown' : 'Dublin 15',
'Blanchardstown' : 'Dublin 15',
'Castleknock' : 'Dublin 15',
'Coolmine' : 'Dublin 15',
'Clonsilla' : 'Dublin 15',
'Corduff' : 'Dublin 15',
'Mulhuddart' : 'Dublin 15',
'Tyrrelstown' : 'Dublin 15',
'Clonee' : 'Dublin 15',
'Ongar' : 'Dublin 15',
'Ballinteer' : 'Dublin 16',
'Ballyboden' : 'Dublin 16',
'Dundrum' : 'Dublin 16',
'Kilmashogue' : 'Dublin 16',
'Knocklyon' : 'Dublin 16',
'Rockbrook' : 'Dublin 16',
'Balgriffin' : 'Dublin 17',
'Coolock' : 'Dublin 17',
'Belcamp' : 'Dublin 17',
'Darndale' : 'Dublin 17',
'Priorswood' : 'Dublin 17',
'Cabinteely' : 'Dublin 18',
'Carrickmines' : 'Dublin 18',
'Foxrock' : 'Dublin 18',
'Kilternan' : 'Dublin 18',
'Sandyford' : 'Dublin 18',
'Shankill' : 'Dublin 18',
'Ticknock' : 'Dublin 18',
'Ballyedmonduff' : 'Dublin 18',
'Stepaside' : 'Dublin 18',
'Leopardstown' : 'Dublin 18',
'Chapelizod' : 'Dublin 20',
'Palmerstown' : 'Dublin 20',
'Clondalkin' : 'Dublin 22',
'Rowlagh' : 'Dublin 22',
'Quarryvale' : 'Dublin 22',
'Liffey Valley' : 'Dublin 22',
'Neilstown' : 'Dublin 22',
'Firhouse' : 'Dublin 24',
'Jobstownand' : 'Dublin 24',
'Tallaght' : 'Dublin 24',
# 'd01' : 'Dublin 1',
# 'd02' : 'Dublin 2',
# 'd03' : 'Dublin 3',
# 'd04' : 'Dublin 4',
# 'd05' : 'Dublin 5',
# 'd06' : 'Dublin 6',
# 'd07' : 'Dublin 7',
# 'd08' : 'Dublin 8',
# 'd09' : 'Dublin 9',
# 'd10' : 'Dublin 10',
# 'd1' : 'Dublin 1',
# 'd2' : 'Dublin 2',
# 'd3' : 'Dublin 3',
# 'd4' : 'Dublin 4',
# 'd5' : 'Dublin 5',
# 'd6' : 'Dublin 6',
# 'd7' : 'Dublin 7',
# 'd8' : 'Dublin 8',
# 'd9' : 'Dublin 9',
# 'd10' : 'Dublin 10',
# 'd12' : 'Dublin 12',
# '1' : 'Dublin 1',
# '2' : 'Dublin 2',
# '3' : 'Dublin 3',
# '4' : 'Dublin 4',
# '5' : 'Dublin 5',
# '6' : 'Dublin 6',
# '6w' : 'Dublin 6W',
# '7' : 'Dublin 7',
# '8' : 'Dublin 8',
# '9' : 'Dublin 9',
# '01' : 'Dublin 1',
# '02' : 'Dublin 2',
# '03' : 'Dublin 3',
# '04' : 'Dublin 4',
# '05' : 'Dublin 5',
# '06' : 'Dublin 6',
# '07' : 'Dublin 7',
# '08' : 'Dublin 8',
# '09' : 'Dublin 9',
# '10' : 'Dublin 10',
# '11' : 'Dublin 11',
# '12' : 'Dublin 12',
# '13' : 'Dublin 13',
# '20' : 'Dublin 20',
}
AreaDict = dict((k.lower(), v.lower()) for k,v in AreaDict.items())
# AreaDict

#### RegExp

In [13]:
def replace(x):
    if not pd.isnull(x):
        x = x.strip()
        tmp = re.search(r'([d]\d{1,2}\s{0,1}[a-z,0-9]{4})', x)
        if tmp:
            return 'dublin ' + tmp[0][2]    
    
        tmp = re.search(r'(0\d{1})', x)
        if tmp:
            return 'dublin ' + tmp[0][1:]
            
        tmp = re.search(r'(dublin\d{1,2})', x)
        if tmp:
            return 'dublin ' + tmp[0][6:]
        
        tmp = re.search(r'(d\d{1,2})', x)
        if tmp:
            return 'dublin ' + tmp[0][1:]

        tmp = re.search(r'(\d{1,2}w*)', x)
        if tmp:
            if tmp[0][0]=='0':
                return 'dublin ' + tmp[0][1:]
            else: 
                return 'dublin ' + tmp[0]
        try: 
            return AreaDict[x].lower() 
        except:
            return np.nan
    
    else: return np.nan

df['street_cleansed'] = df['street_cleansed'].map(lambda x: replace(x))
df.loc[[27514157]]

# df.loc[[72253535]]
# df.head()
# df['street_cleansed'].unique().tolist()
# df[pd.isna(df['street_cleansed'])]

,street,zipcode,description,longitude,latitude,street_cleansed,street_cleansed_from_descr
host_id,,,,,,,
27514157,"cherry field ave lower, dublin 6, dublin, ireland",NaN,"comfortable double in our beautiful 3 bed home in the highly desirable ranelagh area. 2 min walk to top bars and restaurants and 20 walk to city centre. house has modern kitchen, bathroom, lovely period features and private garden. guests will have private double room and use of shared bathroom,sitting room, kitchen and garden someone will be there to greet you on arrival ranelagh is a vibrant neighbourhood minutes walk from dublin city centre. it offers excellent bars, coffee shops and restaurants and is close to some beautiful parks and coastal walks the house is very close to bus,luas and city bike stops so is a great location to explore dublin from",-6.244353,53.32085,NaN,ranelagh


In [14]:
CleanDict = {
 '-' : np.nan, 
 '---': np.nan,
 '+353': np.nan,
#  '0000': np.nan,
#  '0007': 'dublin 7',
#  '001': 'dublin 1',
#  '00353' : np.nan,
#  '008': 'dublin 8',
#  '01 or 07': np.nan,
 'a': np.nan,
 'arran quay' : 'dublin 1',
 'ballygall' : 'dublin 8',
 'botanic' : 'dublin 9',
 'cherry': np.nan,
 'dublin' : np.nan,
 'grace' : np.nan,
 'grange': np.nan,
 'inns quay' : 'dublin 1',
 'kilmore' : np.nan,
 'malahide': np.nan,
 'mountjoy': 'dublin 1',
 'n' : np.nan,
 'na' : np.nan,
 'north' : np.nan,
 'palmerston' : 'dublin 20',
 'phoenix' : 'dublin 8',
 'south dock': np.nan,
 'stillorgan' : np.nan,
 'ushers' : 'dublin 1',
 'xxxx' : np.nan,
 'swords' : np.nan,
 'blackrock': np.nan,
 'dublin 0' : np.nan,   
 'dublin 35' : np.nan,
 'd': np.nan   
}
# CleanDict

In [15]:
# def clean(x):
#     try: 
#         return CleanDict[x] 
#     except:
#         return x
    
# df['street_cleansed'] = df['street_cleansed'].map(lambda x: clean(x))
# df.loc[[72253535]]

In [16]:
df['street_cleansed'].unique()

array(['dublin 4', 'dublin 1', 'dublin 18', 'dublin 2', nan, 'dublin 8',
       'dublin 6', 'dublin 3', 'dublin 11', 'dublin 6w', 'dublin 9',
       'dublin 7', 'dublin 5', 'dublin 12', 'dublin 0', 'dublin 13',
       'dublin 15', 'dublin 14', 'dublin 10', 'dublin 20', 'dublin 35',
       'dublin 17', 'dublin 16', 'dublin '], dtype=object)

In [17]:
# NaN

print(len(df[pd.isnull(df.street_cleansed)]))
df_nan = df[pd.isnull(df.street_cleansed)]
df_nan.columns
df_nan[['street', 'street_cleansed', 'longitude', 'latitude']]

2398


,street,street_cleansed,longitude,latitude
host_id,,,,
103567560,"gardiner street lower, dublin, county dublin, ireland",NaN,-6.257875,53.354777
27514157,"cherry field ave lower, dublin 6, dublin, ireland",NaN,-6.244353,53.320850
73572060,"foley street, dublin, dublin, ireland",NaN,-6.249698,53.353068
758607,"mellowes road, dublin, county dublin, ireland",NaN,-6.312058,53.391034
58390947,"mckee avenue, dublin, dublin, ireland",NaN,-6.300359,53.391177
34508497,"dublin, county dublin, ireland",NaN,-6.238558,53.373433
55770197,"hatch place, dublin, dublin, ireland",NaN,-6.256343,53.333636
3660754,"herberton st, dublin, dublin, ireland",NaN,-6.291733,53.336666
40262548,"synge street, dublin, county dublin, ireland",NaN,-6.267541,53.333928
